
# Propolis Scaffolds — RDKit drawings

This notebook renders 2D structures for:
- **Panel A (top):** exclusive & top scaffolds (⭐) with country labels.
- **Panel B (bottom):** the remaining scaffolds from the **top-10** list (legend of the map), labeled as "Top in N countries".

Inputs (CSV files) expected in the working directory:
- `exclusive_top_scaffolds.csv`  (columns: `country_corrigido`, `scaffold`)
- `top10_scaffolds.csv`          (columns: `scaffold`, `n_countries`)


In [ ]:

# Install (prefer pip official rdkit). If pip fails, consider conda-forge.
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    %pip -q uninstall -y rdkit-pypi >/dev/null
    %pip -q install --upgrade rdkit pillow


In [ ]:

import os, io, pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from PIL import Image as PILImage

# Upload the two CSVs if needed
if 'google.colab' in str(globals()):
    from google.colab import files
    if not os.path.exists("exclusive_top_scaffolds.csv") or not os.path.exists("top10_scaffolds.csv"):
        print("📄 Upload 'exclusive_top_scaffolds.csv' and 'top10_scaffolds.csv'")
        files.upload()

exclusive_df = pd.read_csv("exclusive_top_scaffolds.csv")
top10_df     = pd.read_csv("top10_scaffolds.csv")

# Normalize UK labels (for display only)
country_alias = {
    "England": "United Kingdom",
    "Scotland": "United Kingdom",
    "Wales": "United Kingdom",
    "Northern Ireland": "United Kingdom",
    "UK": "United Kingdom",
    "Great Britain": "United Kingdom",
    "Britain": "United Kingdom",
}
exclusive_df["country_corrigido"] = exclusive_df["country_corrigido"].astype(str).str.strip().replace(country_alias)
exclusive_df = exclusive_df.drop_duplicates(subset=["country_corrigido","scaffold"]).reset_index(drop=True)

def parse_scaffold(s: str):
    if not isinstance(s, str) or not s.strip():
        return None
    mol = Chem.MolFromSmiles(s)
    if mol is None:
        mol = Chem.MolFromSmarts(s)
    if mol is None:
        return None
    try:
        AllChem.Compute2DCoords(mol)
    except Exception:
        pass
    return mol

def to_pil(img_obj):
    if isinstance(img_obj, PILImage.Image):
        return img_obj
    if isinstance(img_obj, (bytes, bytearray)):
        try: return PILImage.open(io.BytesIO(img_obj)).convert("RGBA")
        except: return None
    data = getattr(img_obj, "data", None)
    if isinstance(data, (bytes, bytearray)):
        try: return PILImage.open(io.BytesIO(data)).convert("RGBA")
        except: return None
    return None

def grid_image(mols, legends, per_row, subimg=(360, 290)):
    if not mols: return None, None
    img = Draw.MolsToGridImage(mols, legends=legends, molsPerRow=per_row, subImgSize=subimg, useSVG=False)
    pil = to_pil(img)
    if pil is not None: return pil, None
    svg = Draw.MolsToGridImage(mols, legends=legends, molsPerRow=per_row, subImgSize=subimg, useSVG=True)
    if isinstance(svg, str) and svg.lstrip().startswith("<svg"): return None, svg
    return None, None

# Panel A — exclusive & top
exclusive_mols, exclusive_legends = [], []
for _, row in exclusive_df.iterrows():
    mol = parse_scaffold(str(row["scaffold"]))
    if mol is not None:
        exclusive_mols.append(mol)
        exclusive_legends.append(f"{row['country_corrigido']} — exclusive top")

img1_path_png = "scaffolds_exclusive_top.png"; img1_path_svg = "scaffolds_exclusive_top.svg"
if exclusive_mols:
    per_row = min(4, len(exclusive_mols)) if len(exclusive_mols) else 1
    pil, svg = grid_image(exclusive_mols, exclusive_legends, per_row=per_row, subimg=(380, 300))
    if pil is not None: pil.save(img1_path_png)
    elif svg is not None:
        with open(img1_path_svg, "w", encoding="utf-8") as f: f.write(svg)

# Panel B — remaining of top-10
exclusives_set = set(exclusive_df["scaffold"].astype(str))
top10_df = top10_df.dropna(subset=["scaffold"]).copy()
top10_df["scaffold"] = top10_df["scaffold"].astype(str)
others_df = top10_df[~top10_df["scaffold"].isin(exclusives_set)].copy()

other_mols, other_legends = [], []
for _, r in others_df.iterrows():
    mol = parse_scaffold(r["scaffold"])
    if mol is not None:
        other_mols.append(mol)
        other_legends.append(f"Top in {int(r['n_countries'])} countries")

img2_path_png = "scaffolds_top10_others.png"; img2_path_svg = "scaffolds_top10_others.svg"
if other_mols:
    per_row = min(5, len(other_mols)) if len(other_mols) else 1
    pil2, svg2 = grid_image(other_mols, other_legends, per_row=per_row, subimg=(340, 280))
    if pil2 is not None: pil2.save(img2_path_png)
    elif svg2 is not None:
        with open(img2_path_svg, "w", encoding="utf-8") as f: f.write(svg2)

# 2-panel composite (if PNGs exist)
imgs = []
if os.path.exists(img1_path_png): imgs.append(PILImage.open(img1_path_png))
if os.path.exists(img2_path_png): imgs.append(PILImage.open(img2_path_png))
if imgs:
    maxw = max(im.width for im in imgs); totalh = sum(im.height for im in imgs)
    canvas = PILImage.new("RGB", (maxw, totalh), "white")
    y = 0
    for im in imgs:
        x = (maxw - im.width) // 2; canvas.paste(im, (x, y)); y += im.height
    canvas.save("scaffolds_2panel.png")

# CSV summary
rows = []
for _, r in exclusive_df.iterrows():
    rows.append({"panel":"exclusive_top","country":r["country_corrigido"],"countries_count":1,"scaffold":r["scaffold"]})
for _, r in others_df.iterrows():
    rows.append({"panel":"top10_other","country":"","countries_count":int(r["n_countries"]),"scaffold":r["scaffold"]})
pd.DataFrame(rows).to_csv("scaffolds_render_summary.csv", index=False)

print("Saved files:", [p for p in ["scaffolds_exclusive_top.png","scaffolds_top10_others.png","scaffolds_2panel.png","scaffolds_render_summary.csv","scaffolds_exclusive_top.svg","scaffolds_top10_others.svg"] if os.path.exists(p)])

# Auto-download (Colab)
try:
    if IN_COLAB:
        from google.colab import files
        for f in ["scaffolds_exclusive_top.png","scaffolds_top10_others.png","scaffolds_2panel.png","scaffolds_render_summary.csv","scaffolds_exclusive_top.svg","scaffolds_top10_others.svg"]:
            if os.path.exists(f): files.download(f)
except Exception as e:
    print("Auto-download failed:", e)
